# OpenCV Match

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy
import math
%matplotlib inline

plt.rcParams['figure.figsize'] = [8,6]
plt.rcParams['figure.dpi'] = 120


def imshow(img, pos=None, title=None):
    if isinstance(img, dict):
        imshow_map(img)
        return
    if isinstance(img, list):
        imshow_list(img)
        return
    else:
        if pos != None:
            plt.subplot(pos)
        if (len(img.shape) >=3 and img.shape[2] == 3) or (len(img.shape) == 2):
            plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGBA))
        else:
            plt.imshow(img)
        if title != None:
            plt.title(title)

def imshow_list(lp):
    imshow_map(dict([(idx, value)for idx,value in enumerate(lp)]))


def imshow_map(mp):
    num = len(mp)
    col = 3 if num >= 3 else num % 3 
    row = math.ceil(num / 3)
    idx = 0
    for title in mp:
        img = mp[title]
        idx += 1
        pos = row * 100 + col * 10 + idx
        imshow(img, pos, title)

face = cv2.imread('profile.jpg')
eye = cv2.imread('profile_eye.png')

## 模板匹配
类似卷积原理，模板从原图上开始滑动，计算模板与图象被覆盖的地方的差别程度。
假如原图形是 $A*B$ 大小， 模板是 $a*b$ 大小，则输出结果的矩阵大小是 $(A-a+1)*(B-b+1)$

| method | desc |
|:- |:- |
| TM_SQDIFF | 计算平方不同，计算出来的值越小，越相关 |
| TM_CCORR | 计算相关性，计算出来的值越大越相关 |
| TM_CCOEFF| 计算相关系数，计算出来的值越大越相关 |
| TM_SQDIFF_NORMED | 计算归一化平方不同，计算出来的值越接近 0 ，越相关 |
| TM_CCORR_NORMED | 计算归一化相关性，计算出来的值越接近 1 越相关 |
| TM_CCOEFF_NORMED | 计算归一化相关系数，计算出来的值越接近1，越相关 |

[公式](https://docs.opencv.org/3.3.1/df/dfb/group__imgproc__object.html)

推荐使用带归一化的操作，这样可以更好的判断匹配结果

In [ ]:
res = cv2.matchTemplate(face,eye, cv2.TM_SQDIFF_NORMED)
minval,maxval,minloc,maxloc = cv2.minMaxLoc(res)

canvas = face.copy()
cv2.rectangle(canvas, minloc, (minloc[0] + eye.shape[0], minloc[1] + eye.shape[1]), (0,0,255),3)
imshow(face,221)
imshow(eye,222)
imshow(res,223, "Diff")
imshow(canvas,224)
print(minval)

注意！模板匹配总会返回匹配结果，也就是说匹配不会失败（只是差值会大些），应作进一步判断

In [ ]:
circle = cv2.imread('circle.png')
res = cv2.matchTemplate(face,circle, cv2.TM_SQDIFF_NORMED)
minval,maxval,minloc,maxloc = cv2.minMaxLoc(res)

canvas = face.copy()
cv2.rectangle(canvas, minloc, (minloc[0] + eye.shape[0], minloc[1] + eye.shape[1]), (0,0,255),3)
print(minval)

imshow(face,221)
imshow(eye,222)
imshow(res,223, "Diff")
imshow(canvas,224)

### 匹配多个结果

In [ ]:
face2 = numpy.hstack((face,face))
res = cv2.matchTemplate(face2,eye, cv2.TM_SQDIFF_NORMED)

loc = numpy.where(res < 0.01)

canvas = face2.copy()
for pt in zip(*loc[::-1]):
    minloc = pt
    canvas = cv2.rectangle(canvas, minloc, (minloc[0] + eye.shape[0], minloc[1] + eye.shape[1]), (0,0,255),3)

imshow(face2,221)
imshow(eye,222)
imshow(res,223, "Diff")
imshow(canvas,224)
print(minval)